# Medical Imaging Expert Agent with Strands
In this notebook we create the medical imaging expert agent using the open-source Strands agents framework

#### Install Strands agents and required dependencies

In [ ]:
%pip install strands-agents strands-agents-tools --quiet

#### Ensure the latest version of boto3 is shown below
Ensure the boto3 version printed below is **1.37.1** or higher.

In [ ]:
%pip show boto3

#### Import required libraries

In [ ]:
import boto3
import json
import uuid
import requests
from typing import Dict, Any, List
from strands import Agent, tool
from strands.models import BedrockModel

# Get AWS account information
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
region = boto3.Session().region_name

# Prerequisites

This notebook assumes that you have deployed the CloudFormation stack located at https://github.com/aws-samples/amazon-bedrock-agents-cancer-biomarker-discovery to your AWS account in workshop mode.

#### Setup Lambda function configuration
We'll reuse the existing Lambda function for medical imaging operations

In [ ]:
# Lambda function configuration (reusing existing infrastructure)
medical_imaging_lambda_function_name = "imaging-biomarker-lambda"  # Change if different in your account
medical_imaging_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{medical_imaging_lambda_function_name}"

# Initialize AWS clients
lambda_client = boto3.client('lambda', region_name=region)
bedrock_client = boto3.client('bedrock-runtime', region_name=region)

print(f"Lambda function ARN: {medical_imaging_lambda_function_arn}")
print(f"Region: {region}")
print(f"Account ID: {account_id}")

# Strands Agent Creation
In this section we create the agent using the Strands framework

#### Define agent configuration and instructions

In [ ]:
medical_imaging_agent_name = 'Medical-imaging-expert-strands'
medical_imaging_agent_description = "CT scan analysis using Strands framework"
medical_imaging_agent_instruction = """
You are a medical research assistant AI specialized in processing medical imaging scans of 
patients. Your primary task is to create medical imaging jobs, or provide relevant medical insights after the 
jobs have completed execution. Use only the appropriate tools as required by the specific question. Follow these 
instructions carefully:

1. For computed tomographic (CT) lung imaging biomarker analysis:
   a. Identify the patient subject ID(s) based on the conversation.
   b. Use the compute_imaging_biomarker tool to trigger the long-running job,
      passing the subject ID(s) as an array of strings (for example, ["R01-043", "R01-93"]).
   c. Only if specifically asked for an analysis, use the analyze_imaging_biomarker tool to process the results from the previous job.

2. When providing your response:
   a. Start with a brief summary of your understanding of the user's query.
   b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required.
   c. Present the results of the medical imaging jobs if complete.
"""

#### Define tools for Strands agent
These tools will invoke different services to perform operations for the agent

In [ ]:
def invoke_lambda_function(operation: str, payload: Dict[str, Any] = None) -> Dict[str, Any]:
    """
    Helper function to invoke the existing Lambda function with Bedrock Agent compatible event structure
    """
    if payload is None:
        payload = {}
    
    # Convert subject_id list to JSON string to match Lambda function expectations
    subject_id_list = payload.get('subject_id', [])
    subject_id_value = json.dumps(subject_id_list)
    
    print(f"DEBUG: Original subject_id: {subject_id_list}")
    print(f"DEBUG: JSON string subject_id: {subject_id_value}")
    print(f"DEBUG: Type of JSON string: {type(subject_id_value)}")
    
    # Prepare the event payload to match what the Lambda function expects from Bedrock Agents
    if operation == 'compute_imaging_biomarker':
        event = {
            'actionGroup': 'imagingBiomarkerProcessing',
            'function': 'compute_imaging_biomarker',
            'parameters': [
                {
                    'name': 'subject_id',
                    'type': 'string',  # Changed from 'array' to 'string' since we're sending JSON
                    'value': subject_id_value
                }
            ],
            'sessionAttributes': {},
            'promptSessionAttributes': {}
        }
    elif operation == 'analyze_imaging_biomarker':
        event = {
            'actionGroup': 'imagingBiomarkerProcessing',
            'function': 'analyze_imaging_biomarker',
            'parameters': [
                {
                    'name': 'subject_id',
                    'type': 'string',  # Changed from 'array' to 'string' since we're sending JSON
                    'value': subject_id_value
                }
            ],
            'sessionAttributes': {},
            'promptSessionAttributes': {}
        }
    else:
        raise ValueError(f"Unknown operation: {operation}")
    
    print(f"DEBUG: Event structure being sent: {json.dumps(event, indent=2)}")
    
    try:
        response = lambda_client.invoke(
            FunctionName=medical_imaging_lambda_function_arn,
            InvocationType='RequestResponse',
            Payload=json.dumps(event)
        )
        
        print(f"DEBUG: Lambda invocation successful")
        
        result = json.loads(response['Payload'].read())
        print(f"DEBUG: Lambda response: {json.dumps(result, indent=2)}")
        
        # Extract the actual result from the Bedrock Agent response format
        if 'response' in result and 'responseBody' in result['response']:
            response_body = result['response']['responseBody']
            if 'application/json' in response_body:
                body_content = response_body['application/json']['body']
                
                # Try to parse as JSON if it looks like JSON
                if isinstance(body_content, str):
                    try:
                        if body_content.startswith('{') or body_content.startswith('['):
                            return json.loads(body_content)
                        else:
                            return body_content
                    except json.JSONDecodeError:
                        return body_content
                else:
                    return body_content
        
        return result
        
    except Exception as e:
        print(f"DEBUG: Exception during Lambda invocation: {str(e)}")
        return {"error": str(e)}

# Define the tools using Strands @tool decorator
@tool
def compute_imaging_biomarker(subject_id: List[str]) -> str:
    """
    Compute the imaging biomarker features from lung CT scans within the tumor for a list of patient subject IDs.
    
    Args:
        subject_id (List[str]): An array of strings representing patient subject IDs, example ['R01-222', 'R01-333']
    
    Returns:
        str: Results of the imaging biomarker computation job
    """
    print(f"\nComputing imaging biomarkers for subjects: {subject_id}\n")
    result = invoke_lambda_function('compute_imaging_biomarker', {'subject_id': subject_id})
    print(f"\nComputation Output: {json.dumps(result, indent=2)}\n")
    return json.dumps(result, indent=2)

@tool
def analyze_imaging_biomarker(subject_id: List[str]) -> str:
    """
    Analyze the result imaging biomarker features from lung CT scans within the tumor for a list of patient subject IDs.
    
    Args:
        subject_id (List[str]): An array of strings representing patient subject IDs, example ['R01-222', 'R01-333']
    
    Returns:
        str: Analysis results of the imaging biomarker features
    """
    print(f"\nAnalyzing imaging biomarkers for subjects: {subject_id}\n")
    result = invoke_lambda_function('analyze_imaging_biomarker', {'subject_id': subject_id})
    print(f"\nAnalysis Output: {json.dumps(result, indent=2)}\n")
    return json.dumps(result, indent=2)

# Create list of tools
medical_imaging_tools = [compute_imaging_biomarker, analyze_imaging_biomarker]
print(f"Created {len(medical_imaging_tools)} tools for the Strands agent")

#### Setup AWS Bedrock provider for Strands

In [ ]:
# Create Bedrock model for Strands
model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    region_name=region,
    temperature=0.1,
    streaming=False
)

#### Create the Strands agent

In [ ]:
# Create the Strands agent
try:
    medical_imaging_agent = Agent(
        model=model,
        tools=medical_imaging_tools,
        system_prompt=medical_imaging_agent_instruction
    )
    
    print(f"Successfully created Strands agent: {medical_imaging_agent_name}")
    print(f"Agent has {len(medical_imaging_tools)} tools available")
    
except Exception as e:
    print(f"Error creating agent: {e}")
    raise

#### Test the Strands agent

In [ ]:
# Test the agent with a simple query
test_query = "Can you compute the imaging biomarkers for these 2 patients with patient IDs of R01-083 and R01-040?"

print(f"Testing agent with query: {test_query}")
print("=" * 126)

try:
    # Run the agent
    medical_imaging_agent(test_query)

except Exception as e:
    print(f"Error during agent execution: {e}")
    import traceback
    traceback.print_exc()

#### Advanced usage examples

In [ ]:
# Example of more complex queries
complex_queries = [
    "Compute imaging biomarkers for patients R01-001, R01-002, and R01-003",
    "Analyze the imaging biomarker results for patient R01-083"
]

def test_complex_query(query: str):
    """
    Test a complex query with the agent
    """
    print(f"\nTesting query: {query}")
    print("-" * 120)
    
    try:
        medical_imaging_agent(query)
    except Exception as e:
        print(f"Error: {e}")

for query in complex_queries:
    test_complex_query(query)

#### Session management and conversation continuity

In [ ]:
# Demonstrate conversation continuity
def interactive_session():
    """
    Simple interactive session with the agent
    """
    print("Interactive Medical Imaging Analysis Session")
    print("Type 'quit' to exit")
    print("=" * 100)
    
    while True:
        user_input = input("\n\nYour question: ")
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("Session ended.")
            break
            
        try:
            medical_imaging_agent(user_input)
        except Exception as e:
            print(f"Error: {e}")

interactive_session()

## Summary

This notebook demonstrated how to integrate Strands with a Lambda function.

### Benefits of Strands Approach:
- **Open source** - No vendor lock-in
- **Cost effective** - Only pay for the underlying LLM calls
- **Flexible** - Easy to customize and extend

### Tools Available:
- `compute_imaging_biomarker`: Computes imaging biomarker features from lung CT scans
- `analyze_imaging_biomarker`: Analyzes the imaging biomarker results